<a href="https://colab.research.google.com/github/ad71/ragbot/blob/master/agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>